# YOLO26 vs YOLO11 Benchmark on VisDrone

Independent verification of Ultralytics YOLO26 claims:
- **43% faster CPU inference** (ONNX)
- **Better small object detection** (ProgLoss + STAL)
- **NMS-free end-to-end inference**

Dataset: VisDrone2019-DET (~90% small objects)

Author: [Murat Raimbekov](https://github.com/raimbekovm)

In [ ]:
!pip install -q ultralytics>=8.4.0 pycocotools onnx onnxruntime onnxsim

In [ ]:
import os
import time
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import onnxruntime as ort
from ultralytics import YOLO

print(f"PyTorch {torch.__version__}, CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
CONFIG = {
    'epochs': 50,
    'batch': 16,
    'imgsz': 640,
    'data': 'VisDrone.yaml',
    'device': 0,
    'workers': 4,
}

## Training

In [ ]:
YOLO26_WEIGHTS = 'runs/yolo26n/weights/best.pt'

if os.path.exists(YOLO26_WEIGHTS):
    print(f"Weights exist: {YOLO26_WEIGHTS}, skipping training")
else:
    yolo26 = YOLO('yolo26n.pt')
    yolo26.train(
        data=CONFIG['data'],
        epochs=CONFIG['epochs'],
        batch=CONFIG['batch'],
        imgsz=CONFIG['imgsz'],
        device=CONFIG['device'],
        workers=CONFIG['workers'],
        project='runs',
        name='yolo26n',
        exist_ok=True,
    )

In [ ]:
YOLO11_WEIGHTS = 'runs/yolo11n/weights/best.pt'

if os.path.exists(YOLO11_WEIGHTS):
    print(f"Weights exist: {YOLO11_WEIGHTS}, skipping training")
else:
    yolo11 = YOLO('yolo11n.pt')
    yolo11.train(
        data=CONFIG['data'],
        epochs=CONFIG['epochs'],
        batch=CONFIG['batch'],
        imgsz=CONFIG['imgsz'],
        device=CONFIG['device'],
        workers=CONFIG['workers'],
        project='runs',
        name='yolo11n',
        exist_ok=True,
    )

## Validation

In [ ]:
yolo26_trained = YOLO(YOLO26_WEIGHTS)
yolo11_trained = YOLO(YOLO11_WEIGHTS)

def get_model_info(model):
    n_params = sum(p.numel() for p in model.model.parameters())
    n_layers = len(list(model.model.modules()))
    return n_params, n_layers

params26, layers26 = get_model_info(yolo26_trained)
params11, layers11 = get_model_info(yolo11_trained)
print(f"YOLO26n: {params26/1e6:.2f}M params, {layers26} layers")
print(f"YOLO11n: {params11/1e6:.2f}M params, {layers11} layers")

val26 = yolo26_trained.val(data=CONFIG['data'], imgsz=CONFIG['imgsz'], device=CONFIG['device'])
val11 = yolo11_trained.val(data=CONFIG['data'], imgsz=CONFIG['imgsz'], device=CONFIG['device'])

metrics = pd.DataFrame({
    'Model': ['YOLO26n', 'YOLO11n'],
    'Params_M': [params26/1e6, params11/1e6],
    'mAP50': [val26.box.map50, val11.box.map50],
    'mAP50-95': [val26.box.map, val11.box.map],
    'Precision': [val26.box.mp, val11.box.mp],
    'Recall': [val26.box.mr, val11.box.mr],
})
metrics

## Speed Benchmark

Ultralytics benchmarks CPU speed using ONNX format. We test both GPU (PyTorch) and CPU (ONNX).

In [ ]:
def benchmark_gpu(model, warmup=50, runs=200, imgsz=640):
    dummy = np.random.randint(0, 255, (imgsz, imgsz, 3), dtype=np.uint8)
    for _ in range(warmup):
        model.predict(dummy, device=0, verbose=False)
    torch.cuda.synchronize()
    
    times = []
    for _ in range(runs):
        t0 = time.perf_counter()
        model.predict(dummy, device=0, verbose=False)
        torch.cuda.synchronize()
        times.append((time.perf_counter() - t0) * 1000)
    return np.mean(times), np.std(times)


def benchmark_onnx(onnx_path, warmup=30, runs=100, imgsz=640):
    sess = ort.InferenceSession(onnx_path, providers=['CPUExecutionProvider'])
    input_name = sess.get_inputs()[0].name
    dummy = np.random.rand(1, 3, imgsz, imgsz).astype(np.float32)
    
    for _ in range(warmup):
        sess.run(None, {input_name: dummy})
    
    times = []
    for _ in range(runs):
        t0 = time.perf_counter()
        sess.run(None, {input_name: dummy})
        times.append((time.perf_counter() - t0) * 1000)
    return np.mean(times), np.std(times)

In [ ]:
gpu26, gpu26_std = benchmark_gpu(yolo26_trained)
gpu11, gpu11_std = benchmark_gpu(yolo11_trained)
print(f"GPU - YOLO26n: {gpu26:.2f}±{gpu26_std:.2f}ms, YOLO11n: {gpu11:.2f}±{gpu11_std:.2f}ms")

In [ ]:
onnx26 = yolo26_trained.export(format='onnx', imgsz=CONFIG['imgsz'], simplify=True)
onnx11 = yolo11_trained.export(format='onnx', imgsz=CONFIG['imgsz'], simplify=True)

onnx26_size = os.path.getsize(onnx26) / 1e6
onnx11_size = os.path.getsize(onnx11) / 1e6
print(f"ONNX size - YOLO26n: {onnx26_size:.1f}MB, YOLO11n: {onnx11_size:.1f}MB")

cpu26, cpu26_std = benchmark_onnx(onnx26)
cpu11, cpu11_std = benchmark_onnx(onnx11)
print(f"CPU ONNX - YOLO26n: {cpu26:.2f}±{cpu26_std:.2f}ms, YOLO11n: {cpu11:.2f}±{cpu11_std:.2f}ms")

In [ ]:
speed_df = pd.DataFrame([
    {'Model': 'YOLO26n', 'Device': 'GPU', 'Time_ms': gpu26, 'Std': gpu26_std},
    {'Model': 'YOLO11n', 'Device': 'GPU', 'Time_ms': gpu11, 'Std': gpu11_std},
    {'Model': 'YOLO26n', 'Device': 'CPU-ONNX', 'Time_ms': cpu26, 'Std': cpu26_std},
    {'Model': 'YOLO11n', 'Device': 'CPU-ONNX', 'Time_ms': cpu11, 'Std': cpu11_std},
])

model_df = pd.DataFrame([
    {'Model': 'YOLO26n', 'Params_M': params26/1e6, 'ONNX_MB': onnx26_size},
    {'Model': 'YOLO11n', 'Params_M': params11/1e6, 'ONNX_MB': onnx11_size},
])

cpu_speedup = (1 - cpu26 / cpu11) * 100
print(f"CPU Speedup: YOLO26 is {cpu_speedup:+.1f}% vs YOLO11 (claimed: +43%)")
print(f"\nModel Size:")
print(model_df.to_string(index=False))

## COCO Evaluation by Object Size

Testing ProgLoss + STAL claim for small object detection.

In [ ]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from PIL import Image

CLASSES = ['pedestrian', 'people', 'bicycle', 'car', 'van',
           'truck', 'tricycle', 'awning-tricycle', 'bus', 'motor']
DATA_PATH = Path('datasets/VisDrone')


def create_coco_gt(data_path):
    images_dir = data_path / 'VisDrone2019-DET-val' / 'images'
    labels_dir = data_path / 'VisDrone2019-DET-val' / 'labels'
    
    coco = {'images': [], 'annotations': [], 'categories': [
        {'id': i, 'name': n} for i, n in enumerate(CLASSES)
    ]}
    
    ann_id = 0
    for img_id, img_path in enumerate(sorted(images_dir.glob('*.jpg'))):
        with Image.open(img_path) as im:
            w, h = im.size
        coco['images'].append({'id': img_id, 'file_name': img_path.name, 'width': w, 'height': h})
        
        label_path = labels_dir / f"{img_path.stem}.txt"
        if label_path.exists():
            for line in open(label_path):
                parts = line.strip().split()
                if len(parts) >= 5:
                    cls = int(parts[0])
                    xc, yc, bw, bh = map(float, parts[1:5])
                    xc, yc, bw, bh = xc*w, yc*h, bw*w, bh*h
                    coco['annotations'].append({
                        'id': ann_id, 'image_id': img_id, 'category_id': cls,
                        'bbox': [xc-bw/2, yc-bh/2, bw, bh], 'area': bw*bh, 'iscrowd': 0
                    })
                    ann_id += 1
    return coco


def get_predictions(model, data_path, device=0):
    images_dir = data_path / 'VisDrone2019-DET-val' / 'images'
    preds = []
    for img_id, img_path in enumerate(sorted(images_dir.glob('*.jpg'))):
        results = model.predict(str(img_path), device=device, verbose=False)[0]
        if results.boxes is not None:
            for box, conf, cls in zip(results.boxes.xyxy.cpu().numpy(),
                                       results.boxes.conf.cpu().numpy(),
                                       results.boxes.cls.cpu().numpy()):
                x1, y1, x2, y2 = box
                preds.append({
                    'image_id': img_id, 'category_id': int(cls),
                    'bbox': [float(x1), float(y1), float(x2-x1), float(y2-y1)],
                    'score': float(conf)
                })
    return preds

In [ ]:
gt_dict = create_coco_gt(DATA_PATH)
coco_gt = COCO()
coco_gt.dataset = gt_dict
coco_gt.createIndex()

size_results = []
for name, model in [('YOLO26n', yolo26_trained), ('YOLO11n', yolo11_trained)]:
    preds = get_predictions(model, DATA_PATH)
    coco_dt = coco_gt.loadRes(preds)
    coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()
    size_results.append({
        'Model': name,
        'AP': coco_eval.stats[0],
        'AP_small': coco_eval.stats[3],
        'AP_medium': coco_eval.stats[4],
        'AP_large': coco_eval.stats[5],
    })

size_df = pd.DataFrame(size_results)
size_df

## Results

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 9))
colors = {'YOLO26n': '#2ecc71', 'YOLO11n': '#3498db'}

# mAP
ax = axes[0, 0]
x = np.arange(2)
ax.bar(x - 0.15, metrics['mAP50'], 0.3, label='mAP50', color='#3498db')
ax.bar(x + 0.15, metrics['mAP50-95'], 0.3, label='mAP50-95', color='#2ecc71')
ax.set_xticks(x)
ax.set_xticklabels(metrics['Model'])
ax.set_ylabel('mAP')
ax.set_title('Overall Accuracy')
ax.legend()

# GPU Speed
ax = axes[0, 1]
bars = ax.bar(['YOLO26n', 'YOLO11n'], [gpu26, gpu11], color=[colors['YOLO26n'], colors['YOLO11n']])
ax.set_ylabel('Time (ms)')
ax.set_title('GPU Inference (T4)')
for bar in bars:
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f'{bar.get_height():.1f}', ha='center', va='bottom')

# AP by Size
ax = axes[1, 0]
x = np.arange(3)
ax.bar(x - 0.15, [size_df[size_df['Model']=='YOLO26n'][c].values[0] for c in ['AP_small', 'AP_medium', 'AP_large']], 0.3, label='YOLO26n', color=colors['YOLO26n'])
ax.bar(x + 0.15, [size_df[size_df['Model']=='YOLO11n'][c].values[0] for c in ['AP_small', 'AP_medium', 'AP_large']], 0.3, label='YOLO11n', color=colors['YOLO11n'])
ax.set_xticks(x)
ax.set_xticklabels(['Small', 'Medium', 'Large'])
ax.set_ylabel('AP')
ax.set_title('AP by Object Size')
ax.legend()

# CPU ONNX Speed
ax = axes[1, 1]
bars = ax.bar(['YOLO26n', 'YOLO11n'], [cpu26, cpu11], color=[colors['YOLO26n'], colors['YOLO11n']])
ax.set_ylabel('Time (ms)')
ax.set_title(f'CPU Inference ONNX ({cpu_speedup:+.0f}%)')
for bar in bars:
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f'{bar.get_height():.1f}', ha='center', va='bottom')

plt.suptitle('YOLO26 vs YOLO11 on VisDrone', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('benchmark_results.png', dpi=150)
plt.show()

In [ ]:
small_diff = size_df[size_df['Model']=='YOLO26n']['AP_small'].values[0] - size_df[size_df['Model']=='YOLO11n']['AP_small'].values[0]

print("="*60)
print("SUMMARY")
print("="*60)
print(f"\nModel Size: YOLO26n={params26/1e6:.2f}M, YOLO11n={params11/1e6:.2f}M")
print(f"ONNX Size: YOLO26n={onnx26_size:.1f}MB, YOLO11n={onnx11_size:.1f}MB")
print(f"\nAccuracy (mAP50): YOLO26n={metrics[metrics['Model']=='YOLO26n']['mAP50'].values[0]:.3f}, YOLO11n={metrics[metrics['Model']=='YOLO11n']['mAP50'].values[0]:.3f}")
print(f"Small Object AP: YOLO26n={size_df[size_df['Model']=='YOLO26n']['AP_small'].values[0]:.4f}, YOLO11n={size_df[size_df['Model']=='YOLO11n']['AP_small'].values[0]:.4f} ({small_diff:+.4f})")
print(f"\nGPU Speed (T4): YOLO26n={gpu26:.2f}ms, YOLO11n={gpu11:.2f}ms")
print(f"CPU Speed (ONNX): YOLO26n={cpu26:.2f}ms, YOLO11n={cpu11:.2f}ms ({cpu_speedup:+.1f}%, claimed +43%)")
print("="*60)

In [ ]:
metrics.to_csv('metrics.csv', index=False)
speed_df.to_csv('speed.csv', index=False)
size_df.to_csv('size_metrics.csv', index=False)
model_df.to_csv('model_info.csv', index=False)